# GPT4o code for sentiment labelling (run in colab, downloaded) Initial Version 

Here we design a prompt to request sentiment label from GPT4o-mini model, interatively adjusting our prompts to get desired outputs as we expected.  
You will need to setup library, API Key and load the processed sentence level file to conduct query.


**WARNING:** As syntax of library, model checkpoint updating along the way,
additional edition of this notebook is necessary to reproduce the results!

In [1]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 827.2 kB/s eta 0:00:00


In [1]:
import pandas as pd

data_df = pd.read_csv('/content/sentence_level_script_data_input.csv')
# remove unnamed column
# data_df.drop(columns=['Unnamed: 0'], inplace=True)
# remove none session id
data_df = data_df[~data_df['session_id'].isna()]
# cast session_id into integer
data_df['session_id'] = data_df['session_id'].astype(int)
data_df['speaker_id'] = data_df['speaker_id'].astype(int)
data_df.head()

,file_name,session_id,speaker_id,speaker_type,dialog_type,x_min,x_max,cleaned_text_for_sentiment
0,app_0683_0013_phnd_cc-bnk.TextGrid,683,13,agent,bank,0.00,5.35,hi this is A B C bank how can I help you
1,app_0683_0013_phnd_cc-bnk.TextGrid,683,13,agent,bank,17.15,23.55,ya our bank (uh) do give out ya our bank does ...
2,app_0683_4366_phnd_cc-bnk.TextGrid,683,4366,client,bank,23.28,27.41,[oh] ya probably I can give you some informati...
3,app_0683_4366_phnd_cc-bnk.TextGrid,683,4366,client,bank,27.41,31.26,first (ppb) okay I'm actually having a H_D_B r...
4,app_0683_4366_phnd_cc-bnk.TextGrid,683,4366,client,bank,31.26,38.52,(ppb) (err) (uh) four room flat that I bought ...


In [2]:
data_df.groupby(by=["session_id","dialog_type"])["cleaned_text_for_sentiment"].count().describe()

,cleaned_text_for_sentiment
count,1982.000000
mean,52.223007
std,17.688388
min,10.000000
25%,40.000000
50%,50.000000
75%,62.000000
max,134.000000


## single test

In [11]:
test_data_df = pd.DataFrame({
    'text': [
        "I love this product!",
        "This is terrible.",
        "It was okay, not great.",
        "Absolutely wonderful experience!",
        "Not good, not bad.",
        "Best service ever!",
        "Worst service ever.",
        "I'm quite happy with it.",
        "I'm not satisfied at all.",
        "Could be better, but not too bad."
    ]
})
temp_sentiment = get_session_sentence_sentiment_score_gpt(str(test_data_df["text"].head().to_dict()))
temp_sentiment

,0
7,0.0
8,0.0
11,0.0
15,0.0
19,0.0


### check the stitching

In [24]:
def process_per_dialog(data_df):
    data_df['sentiment_score_GPT'] = -999
    grouped_dialog = data_df.groupby(['session_id', 'dialog_type'])
    count = 0
    result_dfs = []
    for (session_id, dialog_type), dialog_df in grouped_dialog:
        print(f"Processing session_id: {session_id}, dialog_type: {dialog_type}, number of chunks: {dialog_df.shape[0]}")
        input_dict = dialog_df["cleaned_text_for_sentiment"].to_dict()
        num_sentences = dialog_df.shape[0]
        sentiment_scores = pd.Series([count]*num_sentences, index=dialog_df.index)
        # print(sentiment_scores.tail(2).values)
        # sentiment_scores = get_session_sentence_sentiment_score_gpt(str(input_dict))
        # try:
        #     assert len(sentiment_scores) == len(dialog_df)
        # except:
        #     print(f"the dimension not matched for session_id: {session_id}, dialog_type: {dialog_type}, number of chunks: {dialog_df.shape[0]} but get {len(sentiment_scores)}")
        #     continue
        # Reindex the Series to match the DataFrame's index
        aligned_series = sentiment_scores.reindex(data_df.index)
        dialog_df['sentiment_score_GPT'].update(aligned_series) #= sentiment_scores.reset_index(drop=True)
        print("check values after update:",dialog_df['sentiment_score_GPT'].tail(2).values)
        result_dfs.append(dialog_df)
        count+=1
        if count > 5:
            break

    # Concatenate all the processed groups back into a single DataFrame
    result_df = pd.concat(result_dfs)
    return result_df

process_per_dialog(data_df)

Processing session_id: 683, dialog_type: bank, number of chunks: 58
check values after update: [0 0]
Processing session_id: 683, dialog_type: insurance, number of chunks: 75
check values after update: [1 1]
Processing session_id: 683, dialog_type: telecom, number of chunks: 75
check values after update: [2 2]
Processing session_id: 684, dialog_type: bank, number of chunks: 73
check values after update: [3 3]
Processing session_id: 684, dialog_type: insurance, number of chunks: 85
check values after update: [4 4]
Processing session_id: 684, dialog_type: telecom, number of chunks: 95
check values after update: [5 5]


,file_name,session_id,speaker_id,speaker_type,dialog_type,x_min,x_max,cleaned_text_for_sentiment,sentiment_score_GPT
0,app_0683_0013_phnd_cc-bnk.TextGrid,683,13.0,agent,bank,0.00,5.35000,hi this is A B C bank how can I help you,0
1,app_0683_0013_phnd_cc-bnk.TextGrid,683,13.0,agent,bank,17.15,23.55000,ya our bank do give out ya our bank does give ...,0
2,app_0683_4366_phnd_cc-bnk.TextGrid,683,4366.0,client,bank,23.28,27.41000,ya probably I can give you some information on...,0
3,app_0683_4366_phnd_cc-bnk.TextGrid,683,4366.0,client,bank,27.41,31.26000,first okay I'm actually having a H_D_B resale ...,0
4,app_0683_4366_phnd_cc-bnk.TextGrid,683,4366.0,client,bank,31.26,38.52000,four room flat that I bought back in two thous...,0
...,...,...,...,...,...,...,...,...,...
456,app_0684_0001_phnd_cc-tel.TextGrid,684,1.0,agent,telecom,629.61,641.43000,okay can I will process that for you and I wil...,5
457,app_0684_4368_phnd_cc-tel.TextGrid,684,4368.0,client,telecom,652.21,657.47000,okay sure thing yeah,5
458,app_0684_4368_phnd_cc-tel.TextGrid,684,4368.0,client,telecom,657.47,661.13000,nah I gotta go also but thank you so much ya,5
459,app_0684_0001_phnd_cc-tel.TextGrid,684,1.0,agent,telecom,659.37,665.29950,okay ya no worries thank you so much and have ...,5


prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a supply chain expert to help me generate insight from given data below. E-commerce(lazada) platform sales reviews on coke {model} from {start} to {end} are provided below, sales data generated purely in Singapore Market, present in the insights in the following 4 category : 1. sentiment,calculate and show in the {percentage} format. 2. delivery - summarise in text. 3. packaging - summarise in text. 4. improvements_needed - summarise in text ..... please generate the insights and return in JSON format only so it can be used for Front-end Visualization"),
            ("user", "{input}")
        ])


# GPT to assign sentiment score

In [33]:
from openai import OpenAI

client = OpenAI(timeout=30)

## single row sentiment

In [ ]:
# prompt = "You are a skilled Singaporean call center agent to help me generate insights from given data below,"
# prompt += "which are text chunks from inbound calls for banking, insurance, or telecom service inquiries."
# prompt += "The speakers are singpaporeans who often mix special slang during talking."
# prompt += "Please return in only a single number value result. "

# def get_sentiment_score(text):
#     response = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[
#     {"role": "system", "content": prompt},
#     {"role": "user",
#      "content": f"Rate the sentiment of the following on a scale from -1 (negative) to 1 (positive) at 3 decimal places : '{text}'"}
#     ])

#     # Extract the score from the response
#     sentiment_score = response.choices[0].message.content
#     return sentiment_score

# # Sample DataFrame
# data = pd.DataFrame({
#     'text': [
#         "This is an amazing product!",
#         "I am not happy with the service.",
#         "It was okay, not great.",
#         "Absolutely wonderful experience!",
#         "Terrible, will never use again."
#     ]
# })

# # Apply the GPT sentiment labeling function
# data['sentiment_score_GPT'] = data['text'].apply(get_sentiment_score)

# print(data)

                               text sentiment_score_GPT
0       This is an amazing product!                   1
1  I am not happy with the service.                  -1
2           It was okay, not great.                   0
3  Absolutely wonderful experience!                   1
4   Terrible, will never use again.                  -1


## session level all sentences score query

In [39]:
import json
sentiment_system_prompt = '''
You are a skilled singaporean call center agent to help me generate insights
from given text chunks below about sentiment,
They are inbound calls for banking, insurance, or telecom service inquiries,
talking in Singlish with different languange phrases as slangs: Shiok means good, Sian means boring, Jialat means troublesome
Your goal is to rate the sentiment of each text chunk or sentence on a floating number scale from -1 (negative) to 1 (positive)
You will be provided with one phone call dialogue text scripts and you will output a json object containing the following information:

{
    id: float // input chunk index as key & sentiment score as value
}

id should align with the input dictionary
'''
def get_session_sentence_sentiment_score_gpt(description):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
          {
              "role": "system",
              "content": sentiment_system_prompt
          },
          {
              "role": "user",
              "content": description
          }
        ],
        temperature=0.2,
        response_format={ "type": "json_object"},
        timeout=30
        )
    if response:
        sentiment_scores_dict = json.loads(response.choices[0].message.content)
        return pd.Series(sentiment_scores_dict)
    return pd.Series()

# get_session_sentence_sentiment_score_gpt(str(test_data_df["text"].to_dict()))

{\n    "0": 0.5,\n    "1": 0.6,\n    "2": 0.5,\n    "3": 0.4,\n    "4": 0.4\n}

In [ ]:
# def process_sentiment_per_dialog(data_df):
#     data_df['sentiment_score_GPT'] = -999
#     grouped_dialog = data_df.groupby(['session_id', 'dialog_type'])
#     count = 0
#     result_dfs = []
#     result_score_only_series = []
#     failed_session_records = {}
#     for (session_id, dialog_type), dialog_df in grouped_dialog:
#         print(f"Processing session_id: {session_id}, dialog_type: {dialog_type}, number of chunks: {dialog_df.shape[0]}")
#         input_dict = dialog_df["cleaned_text_for_sentiment"].to_dict()
#         sentiment_scores = get_session_sentence_sentiment_score_gpt(str(input_dict))

#         try:
#             assert len(sentiment_scores) == len(dialog_df)
#         except:
#             print(f"the dimension not matched for session_id: {session_id}, dialog_type: {dialog_type}, number of chunks: {dialog_df.shape[0]} but get {len(sentiment_scores)}")
#             failed_session_records = {"session_id": session_id,
#                                       "dialog_type": dialog_type,
#                                       "number of chunks": dialog_df.shape[0],
#                                       "failed_matching_scores": sentiment_scores
#                                       }
#             continue
#         # Reindex the Series to match the DataFrame's index
#         aligned_series = sentiment_scores.reindex(data_df.index)
#         dialog_df['sentiment_score_GPT'].update(aligned_series)
#         # dialog_df['sentiment_score_GPT'] = sentiment_scores.reset_index(drop=True)
#         result_dfs.append(dialog_df)
#         count+=1
#         if count > 5:
#             break

#     # Concatenate all the processed groups back into a single DataFrame
#     result_df = pd.concat(result_dfs)#.reset_index(drop=True)
#     return result_df, failed_session_records

# first_k_result_df,failed_session_records = process_sentiment_per_dialog(data_df)
# first_k_result_df

In [40]:
data_df['sentiment_score_GPT'] = -99.9
grouped_dialog = data_df.groupby(['session_id', 'dialog_type'])
count = 0
result_dfs = []
failed_session_records = []
result_score_only_series = []
for (session_id, dialog_type), dialog_df in grouped_dialog:
    print(f"Processing session_id: {session_id}, dialog_type: {dialog_type}, number of chunks: {dialog_df.shape[0]}")
    # dialog_df = dialog_df[dialog_df]
    input_dict = dialog_df["cleaned_text_for_sentiment"].to_dict()
    if count > 5:
        break
    try:
        sentiment_scores = get_session_sentence_sentiment_score_gpt(str(input_dict))
        sentiment_scores.index = sentiment_scores.index.astype(int)
        result_score_only_series.append(sentiment_scores)
    except Exception as e:
        print(f"Failed at session_id: {session_id}, dialog_type: {dialog_type}")
        failed_session_records.append({"session_id": session_id,
                                  "dialog_type": dialog_type,
                                  "location": "GPT_request",
                                  "reason": str(e)
                                  })
        result_score_only_series.append([])
        count+=1
        continue
    try:
        assert len(sentiment_scores) == len(dialog_df)
    except:
        print(f"the dimension not matched for session_id: {session_id}, dialog_type: {dialog_type}, number of chunks: {dialog_df.shape[0]} but get {len(sentiment_scores)}")
        failed_session_records.append({"session_id": session_id,
                                  "dialog_type": dialog_type,
                                  "location": "result length assert",
                                  "number of chunks": dialog_df.shape[0],
                                  "failed_matching_scores": sentiment_scores
                                  })
        pass
    # Reindex the Series to match the DataFrame's index
    aligned_series = sentiment_scores.reindex(dialog_df.index)
    # print("series values:", aligned_series.values)
    dialog_df['sentiment_score_GPT'].update(aligned_series)
    # print("df col values mean:", dialog_df['sentiment_score_GPT'].mean())
    # dialog_df['sentiment_score_GPT'] = sentiment_scores.reset_index(drop=True)
    result_dfs.append(dialog_df)
    count+=1
result_df = pd.concat(result_dfs)#.reset_index(drop=True)
result_df.tail()

Processing session_id: 683, dialog_type: bank, number of chunks: 58
df col values: [ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.   0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5 -0.2  0.   0.   0.   0.   0.
  0.5  0.   0.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.8  0.5  0.5  0.5  0.5  0.   0.5  0.5  0.5
  0.5  0.5]
Processing session_id: 683, dialog_type: insurance, number of chunks: 75
df col values: [ 0.5  0.5  0.5  0.5  0.7  0.6  0.7  0.7  0.6  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.6  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.4  0.4  0.4
  0.4  0.5 -0.5  0.4 -0.6 -0.5 -0.6  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5 -0.4 -0.4 -0.6  0.5  0.5  0.7  0.6  0.5  0.6  0.5  0.5
  0.6  0.5  0.5  0.4  0.5  0.6  0.5  0.5  0.6  0.7  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.6  0.7]
Processing session_id: 683, dialog_type: telecom, number of chunks: 75
df col values: [ 0.5  0.3  0.4  0.5  0.5  0.5  0.6  0.5  0.5  0.4  0.4  0.3  0.3 

,file_name,session_id,speaker_id,speaker_type,dialog_type,x_min,x_max,cleaned_text_for_sentiment,sentiment_score_GPT
456,app_0684_0001_phnd_cc-tel.TextGrid,684,1,agent,telecom,629.61,641.43000,okay can I will process that for you and I wil...,0.5
457,app_0684_4368_phnd_cc-tel.TextGrid,684,4368,client,telecom,652.21,657.47000,okay sure thing yeah,0.5
458,app_0684_4368_phnd_cc-tel.TextGrid,684,4368,client,telecom,657.47,661.13000,(err) nah I gotta go also but thank you so muc...,0.5
459,app_0684_0001_phnd_cc-tel.TextGrid,684,1,agent,telecom,659.37,665.29950,okay ya no worries thank you so much and have ...,0.5
460,app_0684_4368_phnd_cc-tel.TextGrid,684,4368,client,telecom,661.13,665.37475,alright bye bye,0.5


In [41]:
failed_session_records

[{'session_id': 684,
  'dialog_type': 'insurance',
  'location': 'GPT_request',
  'reason': 'Request timed out.'}]

In [53]:
count = 0
for (session_id, dialog_type), dialog_df in grouped_dialog:
    if count <= 35:
        count += 1
        continue
    if count > 50:
        break
    print(f"Processing session_id: {session_id}, dialog_type: {dialog_type}, number of chunks: {dialog_df.shape[0]}")
    # dialog_df = dialog_df[dialog_df['speaker_type']=='client'] # or other methods to reduce request content
    input_dict = dialog_df["cleaned_text_for_sentiment"].to_dict()
    try:
        sentiment_scores = get_session_sentence_sentiment_score_gpt(str(input_dict))
        sentiment_scores.index = sentiment_scores.index.astype(int)
        result_score_only_series.append(sentiment_scores)
    except Exception as e:
        print(f"Failed at session_id: {session_id}-{dialog_type}")
        failed_session_records.append({"session_id": session_id,
                                  "dialog_type": dialog_type,
                                  "location": "GPT_request",
                                  "reason": str(e)
                                  })
        result_score_only_series.append([])
        count+=1
        continue
    # check the result length same as input
    try:
        assert len(sentiment_scores) == len(dialog_df)
    except:
        print(f"the dimension not matched for session_id: {session_id}, dialog_type: {dialog_type}, number of chunks: {dialog_df.shape[0]} but get {len(sentiment_scores)}")
        failed_session_records.append({"session_id": session_id,
                                  "dialog_type": dialog_type,
                                  "location": "result length assert",
                                  "number of chunks": dialog_df.shape[0],
                                  "failed_matching_scores": sentiment_scores
                                  })
        pass
    # Reindex the Series to match the DataFrame's index
    aligned_series = sentiment_scores.reindex(dialog_df.index)
    dialog_df['sentiment_score_GPT'].update(aligned_series)
    print("check result df col values mean:", dialog_df['sentiment_score_GPT'].mean())
    result_dfs.append(dialog_df)
    count += 1
result_df = pd.concat(result_dfs)#.reset_index(drop=True)
result_df.tail()

Processing session_id: 696, dialog_type: bank, number of chunks: 50
check result df col values mean: 0.09
Processing session_id: 696, dialog_type: insurance, number of chunks: 40
check result df col values mean: 0.35750000000000004
Processing session_id: 696, dialog_type: telecom, number of chunks: 38
check result df col values mean: 0.531578947368421
Processing session_id: 697, dialog_type: bank, number of chunks: 53
check result df col values mean: 0.18867924528301888
Processing session_id: 697, dialog_type: insurance, number of chunks: 65
check result df col values mean: 0.5276923076923077
Processing session_id: 697, dialog_type: telecom, number of chunks: 76
check result df col values mean: 0.5171052631578948
Processing session_id: 698, dialog_type: bank, number of chunks: 74
check result df col values mean: 0.16891891891891891
Processing session_id: 698, dialog_type: insurance, number of chunks: 54
check result df col values mean: 0.09259259259259259
Processing session_id: 698, di

,file_name,session_id,speaker_id,speaker_type,dialog_type,x_min,x_max,cleaned_text_for_sentiment,sentiment_score_GPT
3523,app_0700_0018_phnd_cc-tel.TextGrid,700,18,agent,telecom,489.48919,502.56369,okay so (uh) the plan is per month you need to...,0.5
3524,app_0700_0018_phnd_cc-tel.TextGrid,700,18,agent,telecom,565.67297,573.16075,on top of that right if you have passion card ...,0.5
3525,app_0700_0018_phnd_cc-tel.TextGrid,700,18,agent,telecom,589.54600,600.85569,yup you can do everything online and then you ...,0.5
3526,app_0700_4400_phnd_cc-tel.TextGrid,700,4400,client,telecom,600.61774,609.28294,(mm) okay okay okay [lah] can (um) thank you s...,0.5
3527,app_0700_0018_phnd_cc-tel.TextGrid,700,18,agent,telecom,605.19000,609.28294,okay thank you for your call bye bye,0.5


In [55]:
result_df.to_csv("first50_dialog_sentiment_gpt_score_4Aug.csv",index=False)

### Some checks

In [54]:
len(result_dfs)

50

In [56]:
result_df.shape

(3443, 9)

In [57]:
result_df[["session_id","dialog_type"]].value_counts()

session_id  dialog_type
692         telecom        126
            bank           122
            insurance      118
693         insurance      103
689         telecom        103
693         telecom         98
689         insurance       98
693         bank            97
688         telecom         97
684         telecom         95
686         insurance       90
685         telecom         89
690         insurance       87
686         telecom         82
690         telecom         81
688         insurance       80
690         bank            77
685         bank            77
697         telecom         76
683         insurance       75
            telecom         75
698         bank            74
684         bank            73
691         insurance       72
            telecom         72
688         bank            69
686         bank            67
685         insurance       65
697         insurance       65
691         bank            64
689         bank            63
698         telecom         58
683         bank            58
694         telecom         55
698         insurance       54
697         bank            53
695         insurance       50
696         bank            50
694         bank            41
695         telecom         41
694         insurance       41
696         insurance       40
699         telecom         40
            insurance       39
696         telecom         38
699         bank            37
695         bank            36
700         bank            30
            telecom         29
            insurance       23
Name: count, dtype: int64